!pip install --upgrade requests beautifulsoup4 pandas scikit-learn matplotlib scipy


In [1]:

from bs4 import BeautifulSoup
import requests
import time
import datetime
import pandas as pd

import smtplib


In [2]:
URL_tv = pd.read_csv(r'D:\PortfolioProjects\Amazon_web_Scrapping\amazon_tvs.csv')
URL_cameras = pd.read_csv(r'D:\PortfolioProjects\Amazon_web_Scrapping\amazon_cameras.csv')
URL_laptops = pd.read_csv(r'D:\PortfolioProjects\Amazon_web_Scrapping\amazon_laptops.csv')
URL_mobiles = pd.read_csv(r'D:\PortfolioProjects\Amazon_web_Scrapping\amazon_mobiles.csv')


In [3]:
URL_tv.head(10)

,product,price,rating,rating_count,url
0,Samsung 108 cm (43 inches) Crystal 4K Series U...,"38,990",4.3,"2,271",https://www.amazon.in//gp/slredirect/picassoRe...
1,Samsung 108 cm (43 Inches) Wondertainment Seri...,"32,990",4.3,"8,938",https://www.amazon.in//gp/slredirect/picassoRe...
2,Mi 80 cm (32 inches) HD Ready Android Smart LE...,"16,999",4.3,"54,996",https://www.amazon.in//gp/bestsellers/electron...
3,eAirtec 61 cms (24 inches) HD Ready LED TV 24D...,"7,850",4.0,"5,257",https://www.amazon.in//gp/bestsellers/electron...
4,LG 80 cm (32 inches) HD Ready Smart LED TV 32L...,"17,999",4.3,"10,898",https://www.amazon.in//LG-inches-Ready-Smart-3...
5,OnePlus 80 cm (32 inches) Y Series HD Ready LE...,"15,999",4.2,"23,471",https://www.amazon.in//OnePlus-inches-Ready-An...
6,OnePlus 108 cm (43 inches) Y Series Full HD LE...,"25,999",4.2,"23,471",https://www.amazon.in//OnePlus-inches-Smart-An...
7,Samsung 80 cm (32 inches) Wondertainment Serie...,"19,240",4.3,"8,938",https://www.amazon.in//Samsung-inches-Wonderta...
8,eAirtec 102 cms (40 inches) HD Ready Smart LED...,"15,350",3.9,"1,911",https://www.amazon.in//eAirtec-inches-Ready-Sm...
9,TCL 100 cm (40 inches) Full HD Certified Andro...,"24,999",4.2,"3,941",https://www.amazon.in//TCL-100-3cm-inches-Andr...


In [4]:
print(URL_tv.columns)

Index(['product', 'price', 'rating', 'rating_count', 'url'], dtype='object')


In [16]:
## As expected the data is outdated some of the products are not available or out of the market and the priceing has changed 
## like oneplus 80 cm is now 18,999 rubies
## The ones without price are unavailable and some are not shown because the product has been removed

## Some of the links dont exists anymore, which means the product has been removed from the amazon market.

headers = {"Sec-Ch-Ua": "\"Google Chrome\";v=\"95\", \"Chromium\";v=\"95\", \";Not A Brand\";v=\"99\""}
host = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36"}

url1 = (URL_tv.url).head(10)

for url in url1:
    r = requests.get(url, headers= host)
    soup1 = BeautifulSoup(r.content, "html.parser")
    #soup2 = BeautifulSoup(soup1.prettify(), "html.parser")
    title = soup1.find_all('h1', {'id':'title'})
    for name in title:
        productTitle = name.find('span', {'id':'productTitle'}).text.strip()
        print(productTitle)
    price = soup1.find_all('div', {'id':'price'})
    for cost in price:
        try:
            price = cost.find('span', {'id':'priceblock_ourprice'}).text.strip()
            print(price)
        except:
            if cost.find('span', {'id':'priceblock_dealprice'}).text.strip():
                print(cost.find('span', {'id':'priceblock_dealprice'}).text.strip())
            else:
                cost.find('span', {'id':'priceblock_dealprice'}).text.strip()
                pass

Samsung 108 cm (43 inches) Crystal 4K Series Ultra HD Smart LED TV UA43AUE60AKLXL (Black) (2021 Model)
₹36,990.00
Samsung 108 cm (43 Inches) Wondertainment Series Full HD LED Smart TV UA43TE50AAKXXL (Titan Gray) (2020 model)
LG 80 cm (32 inches) HD Ready Smart LED TV 32LM563BPTC (Dark Iron Gray) (2020 Model)
₹17,499.00
OnePlus 80 cm (32 inches) Y Series HD Ready LED Smart Android TV 32Y1 (Black) (2020 Model)
₹18,999.00
OnePlus 108 cm (43 inches) Y Series Full HD LED Smart Android TV 43Y1 (Black) (2020 Model)
₹27,999.00
Samsung 80 cm (32 inches) Wondertainment Series HD Ready LED Smart TV UA32TE40AAKBXL (Titan Gray) (2020 Model)
₹18,490.00
eAirtec 102 cms (40 inches) HD Ready Smart LED TV 40DJSM (Black) (2020 Model)
₹14,350.00
TCL 100 cm (40 inches) Full HD Certified Android Smart LED TV 40S6500FS (Black) (2020 Model)
₹21,499.00


In [17]:
## There are 7 reviews since some of the product no longer exist (in this case 3)
## I used multiple loops in the try section since there few ways to find revviews on the amazon page but not pricing.
## The rating has not changed except by 0.1 in one case so I would assume the majority has not changed as well
## this does not have to work for others but it works for me

drive = {"Sec-Ch-Ua-Platform": "\"Windows\""}

for url in url1:
    r = requests.get(url, headers = host)
    soup1 = BeautifulSoup(r.content, "html.parser")
    title = soup1.find_all('h1', {'id':'title'})
    for name in title:
        print(name.find('span', {'id':'productTitle'}).text.strip())
        
    reviews = soup1.find_all('div', {'data-hook':'review'})
    try:
        reviews = soup1.find_all('div', {'class':'a-section a-spacing-none a-spacing-top-mini cr-widget-ACR'})
        for star in reviews:
            rating = float(star.find('i', {'data-hook':'average-star-rating'}).text.replace('out of 5 stars','').strip())
            print(rating)
    except:
        reviews = soup1.find_all('div', {'id':'averageCustomerReviews'})
        for star in reviews:
            rating = float(star.find('i', {'class':'a-icon a-icon-star a-star-4-5'}).text.replace('out of 5 stars','').strip())
            print(rating)

Samsung 108 cm (43 inches) Crystal 4K Series Ultra HD Smart LED TV UA43AUE60AKLXL (Black) (2021 Model)
4.2
Samsung 108 cm (43 Inches) Wondertainment Series Full HD LED Smart TV UA43TE50AAKXXL (Titan Gray) (2020 model)
4.3
LG 80 cm (32 inches) HD Ready Smart LED TV 32LM563BPTC (Dark Iron Gray) (2020 Model)
4.3
OnePlus 80 cm (32 inches) Y Series HD Ready LED Smart Android TV 32Y1 (Black) (2020 Model)
4.2
OnePlus 108 cm (43 inches) Y Series Full HD LED Smart Android TV 43Y1 (Black) (2020 Model)
4.2
Samsung 80 cm (32 inches) Wondertainment Series HD Ready LED Smart TV UA32TE40AAKBXL (Titan Gray) (2020 Model)
4.3
eAirtec 102 cms (40 inches) HD Ready Smart LED TV 40DJSM (Black) (2020 Model)
3.9
TCL 100 cm (40 inches) Full HD Certified Android Smart LED TV 40S6500FS (Black) (2020 Model)
4.2


In [19]:
## This is customer reviews with their given rating of the product and title of the comments

for url in url1:
    r = requests.get(url, headers = host)
    soup1 = BeautifulSoup(r.content, "html.parser")
    title = soup1.find_all('h1', {'id':'title'})
    for name in title:
        print(name.find('span', {'id':'productTitle'}).text.strip())
        
    reviews = soup1.find_all('div', {'data-hook':'review'})
    for star in reviews:
        rating = float(star.find('i', {'data-hook':'review-star-rating'}).text.replace('out of 5 stars','').strip())
        print(rating)
        
    CustomerReview = soup1.find_all('div', {'data-hook':'review'})
    for item in CustomerReview:
        reviewtitle = item.find('a', {'data-hook':'review-title'}).text.strip()
        print(reviewtitle)

Samsung 108 cm (43 inches) Crystal 4K Series Ultra HD Smart LED TV UA43AUE60AKLXL (Black) (2021 Model)
5.0
1.0
5.0
1.0
5.0
1.0
1.0
1.0
Mindblowing TV
Worst quality check and customer
Worth every single Penny.
Not worth it..
Changed my MI TV, and back to quality
TV screen has become defective within 1 month of purchase from Amazon.  Don't buy from Amazon.
Tv is good but I got open product
No one came or called for installation.
Samsung 108 cm (43 Inches) Wondertainment Series Full HD LED Smart TV UA43TE50AAKXXL (Titan Gray) (2020 model)
1.0
1.0
1.0
1.0
1.0
1.0
4.0
1.0
does not support bluetooth technology
Pathetic Excuses
Need to change to discrimination off product
Never buy electronics from AMAZON
Defective, Low quality and Pathetic
not good
A very good upgrade but FULL HD resolution is missing given the price point of around 17000.
Not worth for money
LG 80 cm (32 inches) HD Ready Smart LED TV 32LM563BPTC (Dark Iron Gray) (2020 Model)
1.0
1.0
5.0
5.0
1.0
4.0
5.0
One major problem on 

In [20]:
## Reviews of each Amazon member commented on each product

for url in url1:
    r = requests.get(url, headers = host)
    soup1 = BeautifulSoup(r.content, "html.parser")
    title = soup1.find_all('h1', {'id':'title'})
    for name in title:
        print(name.find('span', {'id':'productTitle'}).text.strip())
        
    reviews = soup1.find_all('div', {'data-hook':'review'})
    for body in reviews:
        textReview = body.find('span', {'data-hook':'review-body'}).text.strip()
        print(textReview)
        
    

Samsung 108 cm (43 inches) Crystal 4K Series Ultra HD Smart LED TV UA43AUE60AKLXL (Black) (2021 Model)
Pros:1) Picture and sound Quality is next level...2) Universal Remote3) Bezel less screenCons:1) Plastic body2) No Considerable difference b/w 4k and 4k pro model3) TV Thickness is more, feels bulky

Read more
Your browser does not support HTML5 video.
        


   My brand new tv switch offs abruptly within 1 min from switching on., unable to raise replacement query due to poor back end support

Read more
Everything about this is great. Only con is the plastic body.

Read more
Not worth 49k. Having 50inch crystle 4k. Many features are disabled as default. Picture quality is good but you can get the same quality in much less priced tv. Sound is pathetic. Even at 50, it is too less. And the worst part is that it is too laggy. Even apps takes ages to load. Only 1.5 ram and 8 GB storage. But you will only get 2.3 gb storage. Writing by personal experience thay you will regret afterwards

In [244]:
import csv
header = ['title', 'price', 'rating']
url1 = (URL_tv.url)
with open('AmazonWebScrapper.csv','w', newline = '', encoding = ' utf8') as f:
    writer = csv.writer(f)
    writer.writerow(header)

    for url in url1:    
            r = requests.get(url, headers= headers)
            soup1 = BeautifulSoup(r.content, "html.parser")
            #soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

            title = soup1.find_all('h1', {'id':'title'})
            for name in title:
                productTitle = name.find('span', {'id':'productTitle'}).text.strip()
            
            price = soup1.find_all('div', {'id':'price'})
            for cost in price:
                try:
                    price = cost.find('span', {'id':'priceblock_ourprice'}).text.strip()
                except:
                    if cost.find('span', {'id':'priceblock_dealprice'}).text.strip():
                        price = cost.find('span', {'id':'priceblock_dealprice'}).text.strip()
                    else:
                        price = cost.find('span', {'id':'priceblock_dealprice'}).text.strip()
                        pass
            
            reviews = soup1.find_all('div', {'data-hook':'review'})
            try:
                reviews = soup1.find_all('div', {'class':'a-section a-spacing-none a-spacing-top-mini cr-widget-ACR'})
                for star in reviews:
                    rating = star.find('i', {'data-hook':'average-star-rating'}).text.replace('out of 5 stars','').strip()
            except:
                reviews = soup1.find_all('div', {'id':'averageCustomerReviews'})
                for star in reviews:
                    rating = float(star.find('i', {'class':'a-icon a-icon-star a-star-4-5'}).text.replace('out of 5 stars','').strip())
            
            brand = [productTitle, price, rating]
            writer.writerow(brand)


